In [1]:
!pip install coremltools

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import coremltools as ct
import tensorflow as tf

print("TF Version: ", tf.__version__)
print("Eager mode enabled: ", tf.executing_eagerly())
print("GPU available: ", tf.test.is_gpu_available())

TF Version:  1.14.0
Eager mode enabled:  False
GPU available:  False


In [3]:
original_predict_network = 'gs://mobile-ml-wg/arbitrary_style_transfer/original/SavedModel/predict'
original_stylize_network = 'gs://mobile-ml-wg/arbitrary_style_transfer/original/SavedModel/stylize'
original_transfer_network = 'gs://mobile-ml-wg/arbitrary_style_transfer/original/SavedModel/transfer'

small_predict_network = 'gs://mobile-ml-wg/arbitrary_style_transfer/small/SavedModel/predict'
small_stylize_network = 'gs://mobile-ml-wg/arbitrary_style_transfer/small/SavedModel/stylize'
small_transfer_network = 'gs://mobile-ml-wg/arbitrary_style_transfer/small/SavedModel/transfer'

In [4]:
def print_model_details(saved_model_path):
    print('\n{} model'.format(saved_model_path))
    model = tf.compat.v2.saved_model.load(saved_model_path)
    print(model.tensorflow_version)
    for input in model.signatures['serving_default'].inputs:
        print(input)
    print(model.signatures['serving_default'].structured_outputs)

In [5]:
# print_model_details(original_predict_network)
# print_model_details(original_stylize_network)
# print_model_details(original_transfer_network)

# print_model_details(small_predict_network)
# print_model_details(small_stylize_network)
# print_model_details(small_transfer_network)

In [6]:
def convert_tflite_model(saved_model_path, tflite_path, supported_types=[]):
    print('\n{} model'.format(saved_model_path))
    model = tf.compat.v2.saved_model.load(saved_model_path)
    concrete_func = model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
    for input in concrete_func.inputs:
        if input.name == 'style_image:0':
            input.set_shape([1, 256, 256, 3])
        elif input.name == 'content_image:0' and len(supported_types) > 0:
            input.set_shape([1, 384, 384, 3])
        elif input.name == 'mobilenet_conv/Conv/BiasAdd:0' and len(supported_types) > 0:
            input.set_shape([1, 1, 1, 100])

    converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
    converter.optimizations = [tf.lite.Optimize.DEFAULT]

    if len(supported_types) > 0:
        converter.target_spec.supported_types = supported_types

    tflite_model = converter.convert()

    with tf.io.gfile.GFile(tflite_path, 'wb') as f:
        f.write(tflite_model)

    print('Quantized model:', tflite_path, 
        'Size:', len(tflite_model) / 1024, "kb")

In [7]:
# convert_tflite_model(original_predict_network, 'original_predict.tflite')
# convert_tflite_model(original_stylize_network, 'original_stylize.tflite')
# convert_tflite_model(original_transfer_network, 'original_transfer.tflite')

# convert_tflite_model(original_predict_network, 'original_predict_f16.tflite', [tf.float16])
# convert_tflite_model(original_stylize_network, 'original_stylize_f16.tflite', [tf.float16])
# convert_tflite_model(original_transfer_network, 'original_transfer_f16.tflite', [tf.float16])

In [8]:
def print_tflite_model_details(tflite_path):
    print('\n{} model'.format(tflite_path))
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    inputs = interpreter.get_input_details()
    # print('{} inputs:'.format(len(inputs)))
    for i in range(0, len(inputs)):
        print('{} {} {}'.format(inputs[i]['name'], inputs[i]['shape'], inputs[i]['dtype']))

    outputs = interpreter.get_output_details()
    # print('{} outputs:'.format(len(outputs)))
    for i in range(0, len(outputs)):
        print('{} {} {}'.format(outputs[i]['name'], outputs[i]['shape'], outputs[i]['dtype']))

In [9]:
# print_tflite_model_details('original_predict.tflite')
# print_tflite_model_details('original_stylize.tflite')
# print_tflite_model_details('original_transfer.tflite')

# print_tflite_model_details('original_predict_f16.tflite')
# print_tflite_model_details('original_stylize_f16.tflite')
# print_tflite_model_details('original_transfer_f16.tflite')

In [10]:
# hub_network = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'

In [11]:
def convert_predict_model(network):
    saved_model_path = '{}/SavedModel/predict'.format(network)
    coreml_path = '{}_style_predict.mlmodel'.format(network)
    print(saved_model_path)

    mlmodel = ct.convert(saved_model_path,
                         source='tensorflow',
                         inputs=[ct.ImageType(bias=[0,0,0], scale=1/255.0)])
    print(mlmodel)
    mlmodel.save(coreml_path)
    
def convert_transfer_model(network):
    saved_model_path = '{}/SavedModel/transfer'.format(network)
    coreml_path = '{}_style_transfer.mlmodel'.format(network)
    print(saved_model_path)

    mlmodel = ct.convert(saved_model_path,
                         source='tensorflow',
                         inputs=[ct.TensorType(shape=(1, 1, 1, 100)),
                                 ct.ImageType(bias=[0,0,0], scale=1/255.0)])
    print(mlmodel)
    mlmodel.save(coreml_path)

In [12]:
convert_predict_model('original')
convert_transfer_model('original')

original/SavedModel/predict


Translating MIL ==> MLModel Ops: 100%|██████████| 1075/1075 [00:00<00:00, 1113.95 ops/s]


input {
  name: "style_image"
  type {
    imageType {
      width: 1
      height: 1
      colorSpace: RGB
      imageSizeRange {
        widthRange {
          lowerBound: 1
          upperBound: -1
        }
        heightRange {
          lowerBound: 1
          upperBound: -1
        }
      }
    }
  }
}
output {
  name: "Conv/BiasAdd"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "tensorflow==1.14.0"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "4.1"
  }
}

original/SavedModel/transfer


ValueError: NodeDef expected inputs 'float, int32' do not match 1 inputs specified; Op<name=ExpandDims; signature=input:T, dim:Tdim -> output:T; attr=T:type; attr=Tdim:type,default=DT_INT32,allowed=[DT_INT32, DT_INT64]>; NodeDef: {{node transformer/residual/residual1/conv1/StyleNorm/ExpandDims}}